# Using the Omeka S API



In [1]:
import json
import requests

Set up the `get_credential()` function:

In [2]:
def get_credential(json_file, key, sub_key):
   try:
       with open(json_file) as f:
           data = json.load(f)
           return data[key][sub_key]
   except Exception as e:
       print("Error: ", e)

In [14]:
baseURL = 'http://jajohnst.projectst.si.umich.edu/omekas/api/'
key_identity = get_credential('secrets.json', 'omekas', 'key_identity')
key_credential = get_credential('secrets.json', 'omekas', 'key_credential') 

Create a dictionary of the keys, that you can reuse:

In [4]:
omekas_credentials = {
    'key_identity': key_identity,
    'key_credential': key_credential
}

## Omeka S Metadata available through the API

Omeka S stores various levels of information about items in its collection:

* __Item Metadata__ (about an individual item, usually associated with an image or other media file)
* __Multimedia__ (files for media that are linked to item metadata, often still images but may also be moving images, audio, datafiles, or other digital objects)
* __Item Sets__ (items may be grouped into one or more sets that can be used to group or sort items for exhibit purposes)

### Get metadata information for a single item

Items have unique IDs that are created upon ingest. All information about an item
may be retrieved using that ID, and from the item endpoint. The item endpoint
for this repository is

```url
http://jajohnst.projectst.si.umich.edu/omekas/api/items/:item_id
```

Query that endpoint to get metadata for item 82:

In [5]:
resource_type = 'items'
item_ID = 587

In [15]:
item_request = requests.get(baseURL + resource_type + '/' + str(item_ID), params=omekas_credentials)

In [16]:
item_request.url

'http://jajohnst.projectst.si.umich.edu/omekas/api/items/587?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875'

In [17]:
item_request.status_code

200

In [18]:
item_request.json()

{'@context': 'http://jajohnst.projectst.si.umich.edu/omekas/api-context',
 '@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/items/587',
 '@type': ['o:Item', 'dctype:Image'],
 'o:id': 587,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/users/2',
  'o:id': 2},
 'o:resource_class': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/resource_classes/26',
  'o:id': 26},
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'Main public library building, Detroit, Michigan. Cass Gilbert, architect. Postcard by Albertype Co., 1921-1930. Prints & Photographs Division',
 'thumbnail_display_urls': {'large': 'http://jajohnst.projectst.si.umich.edu/omekas/files/large/52d9101c9e01f8841edea69d21d2f802c009d16c.jpg',
  'medium': 'http://jajohnst.projectst.si.umich.edu/omekas/files/medium/52d9101c9e01f8841edea69d21d2f802c009d16c.jpg',
  'square': 'http://jajohnst.projectst.si.umich.edu/omekas/files/square/52d9101c9e01f8841edea69d21

In [19]:
item_metadata = item_request.json()

In [20]:
item_metadata['o:id']

587

## Determine the Context

What is the `@context` tag referring to? This is key to understanding 
the prefixes for many of the elements that you see in these JSON responses.

In [26]:
item_request.json()['@context']

'http://jajohnst.projectst.si.umich.edu/omekas/api-context'

Omeka S uses JSON-LD. This allows the reference to
specific metadata schemes using abbreviated prefixes, 
akin to the namespace process in XML. For example, in the
above records, many terms are prefixed with `o:` or `dcterms:`.
The "context," or namespace map for the JSON-LD, 
shows which 
namespaces are being used in the site's metadata. 
This map is available at the above-referenced URL,
http://jajohnst.projectst.si.umich.edu/omekas/api-context.

To view it:

In [27]:
context_r = requests.get('http://jajohnst.projectst.si.umich.edu/omekas/api-context')

In [28]:
context_r.json()

{'@context': {'o': 'http://omeka.org/s/vocabs/o#',
  'dcterms': 'http://purl.org/dc/terms/',
  'dctype': 'http://purl.org/dc/dcmitype/',
  'bibo': 'http://purl.org/ontology/bibo/',
  'foaf': 'http://xmlns.com/foaf/0.1/',
  'mods': 'http://www.loc.gov/mods/rdf/v1',
  'o-cnt': 'http://www.w3.org/2011/content#',
  'o-time': 'http://www.w3.org/2006/time#'}}

## What item sets are there? 

### What is an item set? 

Omeka has three major resource types: `item` (the individual items in the collection, generally as represented by metadata), `item_sets` (the collections of items and they're grouped together), `media` (files like images that represent the content of individual items).

### So, can you get a list of the item sets?

How can you get a list of what item types are in your instance of Omeka? 

To request the list, you need to identify for Omeka what sort of resource you want to know about. 
To do this, add the resource type to the API endpoint request. The pattern 
for this request is listed at [https://omeka.org/s/docs/developer/api/rest_api/#read](https://omeka.org/s/docs/developer/api/rest_api/#read). 

In [29]:
r = requests.get(baseURL + 'item_sets', params=omekas_credentials)

r.status_code

200

In [30]:
for set in r.json():
    print(set['o:id'], ':', set['o:title'])

1 : Test imported from CSV
145 : Shiba Inus
146 : Free to Use Libraries
148 : Inbox for new Import groups
150 : NewItemSetfromAPI
151 : NewItemSetfromAPI


## Determining errors and addressing them

This method of querying the API can be frustrating when it doesn't respond in the expected way. Say, for exmaple, that the request is not asking for a valid resource type. Knowing that we don't receive the expected response is enough to indicate an error, but it doesn't explain how to correct the error. In these cases, you can check the `status_code` property of the response, and inspect the Omeka JSON response. 

Below, the API was queried for `item_set` rather than `item_sets`:

In [31]:
r = requests.get(baseURL + 'item_set', params={'key_credential': key_credential, 'key_identity': key_identity})

r.status_code

500

The HTTP 500 code indicates a server error, but what caused it? 

The content of the response can be displayed with the `text` method: 

In [32]:
r.text

'{"errors":{"error":"The API does not support the \\u0022item_set\\u0022 resource."}}'

It looks like JSON, so the `json` method will be more helpful:

In [33]:
r.json()['errors']['error']

'The API does not support the "item_set" resource.'

After further research in the [Omeka API reference](https://omeka.org/s/docs/developer/api/api_reference/#parameters-for-item_sets), it seems that in this case `item_sets` must be plural. Thus, the following URL will produce a successful response when the appropriate security keys are appended: 

```
http:// + <URL endpoint> + /api/item_sets/ + <additional URL parameters>
```

## List an Item Set

Get the list of all items in a known item set. In the example below, 
the query asks for ID `146`, but your site's collections will likely have different ID sequences. 
After it is retrieved, check for the name of the set.  

In [34]:
api_resource = 'item_sets'
item_set_id = 146

In [36]:
r = requests.get(baseURL + api_resource + '/' + str(item_set_id), params=omekas_credentials)

print(r.status_code)

200


In [37]:
r.url

'http://jajohnst.projectst.si.umich.edu/omekas/api/item_sets/146?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875'

In [38]:
r.headers

{'Date': 'Mon, 06 Nov 2023 05:32:41 GMT', 'Server': 'Apache', 'Omeka-S-Version': '3.2.3', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8'}

In [39]:
r.json()

{'@context': 'http://jajohnst.projectst.si.umich.edu/omekas/api-context',
 '@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/item_sets/146',
 '@type': 'o:ItemSet',
 'o:id': 146,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/users/1',
  'o:id': 1},
 'o:resource_class': None,
 'o:resource_template': None,
 'o:thumbnail': None,
 'o:title': 'Free to Use Libraries',
 'thumbnail_display_urls': {'large': 'http://jajohnst.projectst.si.umich.edu/omekas/files/large/20fc57b62847f755144857d46104d980e7a42995.jpg',
  'medium': 'http://jajohnst.projectst.si.umich.edu/omekas/files/medium/20fc57b62847f755144857d46104d980e7a42995.jpg',
  'square': 'http://jajohnst.projectst.si.umich.edu/omekas/files/square/20fc57b62847f755144857d46104d980e7a42995.jpg'},
 'o:created': {'@value': '2022-10-19T01:22:10+00:00',
  '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
 'o:modified': {'@value': '2022-10-19T01:22:10+00:00',
  '@type': 'http://www.w3.org/200

What is the name of the set? Look for a `title` element . . . 

In [40]:
title = r.json()['o:title']

print(title)

Free to Use Libraries


## List items in an item set

How can you get a list of all the items in a particular set? 
Continuing with the previous example, let's do this for item set `146`. 

Now, prepare the information for the API request. This requires changing
the resource type for the endpoint and updating the parameters dictionary 
to add all the required variables for the request. (The resource type is changed to `item` for the endpoing, and the `item_set_id` is added to the parameters.) 

In [41]:
api_resource = 'items'
omekas_credentials['item_set_id'] = 146

In [42]:
r = requests.get(baseURL + api_resource, params=omekas_credentials)

In [43]:
r.status_code

200

In [44]:
r.url

'http://jajohnst.projectst.si.umich.edu/omekas/api/items?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146'

In [45]:
for item in r.json():
    print(item['o:id'], item['o:title'])

78 Carnegie Library, Cordele, Georgia
79 Carnegie Library, Sheldon, Iowa
80 Curb service 10,000 current books - convenient, free, time saving : Chicago Public Library, Randolph St. corridor.
81 For greater knowledge on more subjects use your library more often
82 [Cossitt Library, Memphis, Tenn.]
83 Little Rock Public Library, Little Rock, Ark.
84 Buffalo, N.Y. -  Public library - children's room
85 Main public library building - Detroit
86 Newspaper reading room of public library. Omaha, Nebraska
87 Public library in the southwestern section of Mississippi, in the piney woods
88 Library tent at the FSA (Farm Security Administration) mobile camp for migratory farm workers. Odell, Oregon. The girls working in the library receive credit in the Junior Campers League for work in the library
89 Daytona Beach, Florida. Bethune-Cookman College. Students in the library reading room
90 Libraries, Washington D.C.
91 Public Library, Salt Lake City, Utah
92 George Peabody Library, formerly the Lib

## Saving the item information

The Omeka S API can be used to modify or update records, but because of the way that it makes such changes, it is useful to download and store locally the information. 
This should generally serve as a temporary local data store, which can be deleted later once updates are completed. 
So, create a local copy of the items.

### First, create a list of the desired item IDs

The below uses a list comprehension, which is a more compact way to create a list. In the below, the append function is assumed for each result of the iteration in the `for` loop. 

In [46]:
item_ids = [item['o:id'] for item in r.json()]

Nota bene (NB):

The above could also be written like this:

```python
item_ids = list()

for item in r.json():
    item_ids.append(item['o:id'])
```

However the list is created, you should be able to confirm it is a list (and in this case I know it has, or at least expect it to have, 25 items).

In [47]:
print(f'item_ids is a {type(item_ids)} with {len(item_ids)} items.')

item_ids is a <class 'list'> with 25 items.


### Second, retrieve and save the data for each item

This uses the previously created list as a source for the desired items. 
Using the API, it is possible to request the metadata for each item. 
Later, this will be the source for our changes or updates. 

For this step, construct an API endpoint that will query each item:

In [50]:
api_resource = 'items'

# test request
r_test_item = requests.get(baseURL + api_resource + '/' + str(item_ids[0]), params=omekas_credentials)

In [51]:
r_test_item.status_code

200

In [52]:
r_test_item.json()

{'@context': 'http://jajohnst.projectst.si.umich.edu/omekas/api-context',
 '@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/items/78',
 '@type': ['o:Item', 'dctype:Image'],
 'o:id': 78,
 'o:is_public': True,
 'o:owner': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/users/2',
  'o:id': 2},
 'o:resource_class': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/resource_classes/26',
  'o:id': 26},
 'o:resource_template': {'@id': 'http://jajohnst.projectst.si.umich.edu/omekas/api/resource_templates/2',
  'o:id': 2},
 'o:thumbnail': None,
 'o:title': 'Carnegie Library, Cordele, Georgia',
 'thumbnail_display_urls': {'large': 'http://jajohnst.projectst.si.umich.edu/omekas/files/large/20fc57b62847f755144857d46104d980e7a42995.jpg',
  'medium': 'http://jajohnst.projectst.si.umich.edu/omekas/files/medium/20fc57b62847f755144857d46104d980e7a42995.jpg',
  'square': 'http://jajohnst.projectst.si.umich.edu/omekas/files/square/20fc57b62847f755144857d46104d980e7a42995.jpg'

This looks like it is successfully retrieving each item, so to save them locally, it should be possible to retrieve each item, append it to a list, and save the list as a local JSON file. 

Make a list called `records` to store these items:

In [54]:
records = list()

for item in item_ids:
    item = requests.get(baseURL + 'items/' + str(item), params=omekas_credentials)
    if item.status_code != 200:
        print(f'Error: your request for {str(item)} was not successful. HTML response code: {item.status_code}')
        continue
    else:
        print('retrieving', item.url)
        item_metadata = item.json()
        records.append(item_metadata)

retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/78?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146
retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/79?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146
retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/80?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146
retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/81?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146
retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/82?key_identity=JwGow8saqFgZlrmsA5dXwlqVB4XvrhWa&key_credential=FYbJFHUIsEbFjfKDioR70kvuX21le875&item_set_id=146
retrieving http://jajohnst.projectst.si.umich.edu/omekas/api/items/83?key_identity=JwGow8saqFgZlrmsA

In [55]:
len(records)

25

In [56]:
for item in records:
    print(item['o:id'])

78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
112
113
114
115
116
117
118
119


Great. Now, write the metadata to a file.

In [57]:
fname = api_resource + '_set_' + str(item_set_id) + '.json'

In [58]:
with open(fname, 'w', encoding='utf-8') as f:
    json.dump(records, f, indent=2)
    print(f'wrote {len(item_metadata)} items to {fname}')

wrote 25 items to items_set_146.json


To explore this data locally, open and parse that file using the `json` module.

Or, if you are feeling ambitious, you may want to start exploring the `pyld` 
module, which provides some additional functions for working with JSON-LD. 

## Making a Change using the API

Your user will require update permissions to do this. 

In [62]:
modification = open('data/json/omeka-s-title-example.json')

In [63]:
modification.close()